In [628]:
!pip install snowflake-snowpark-python

In [629]:

!pip install snowflake-connector-python[pandas]

In [630]:
!pip install cachetools

In [631]:
from snowflake.connector.pandas_tools import write_pandas

In [632]:
from snowflake.snowpark.session import Session
import pandas as pd
housing=pd.read_csv("housing_dataset.csv")

connection_parameter={
     "account": "xw90760.ap-southeast-1",
    "user": "MUNEERSAYYED",
    "password": "Muneer@786",  # Ensure the password is correct and secure
    "database": "SNOWFLAKE_MUNEER",
    "schema": "TEST_SCHEMA",
    "warehouse": "compute_wh",
    "role": "Accountadmin"   
}
session=Session.builder.configs(connection_parameter).create()
session1=Session.builder.configs(connection_parameter).create()

In [633]:
query = """CREATE OR REPLACE TABLE HOUSING_DATA (
    LONGITUDE FLOAT,
    LATITUDE FLOAT,
    HOUSING_MEDIAN_AGE FLOAT,
    TOTAL_ROOMS FLOAT,
    TOTAL_BEDROOMS FLOAT,
    POPULATION FLOAT,
    HOUSEHOLDS FLOAT,
    MEDIAN_INCOME FLOAT,
    MEDIAN_HOUSE_VALUE FLOAT,
    OCEAN_PROXIMITY VARCHAR
)"""
session.sql(query).collect()

[Row(status='Table HOUSING_DATA successfully created.')]

In [634]:
housing.columns=[x.upper() for x in housing.columns]

In [635]:
housing.head()

,LONGITUDE,LATITUDE,HOUSING_MEDIAN_AGE,TOTAL_ROOMS,TOTAL_BEDROOMS,POPULATION,HOUSEHOLDS,MEDIAN_INCOME,MEDIAN_HOUSE_VALUE,OCEAN_PROXIMITY
0,-122.23,37.88,41,880,129.0,322,126,8.3252,452600,NEAR BAY
1,-122.22,37.86,21,7099,1106.0,2401,1138,8.3014,358500,NEAR BAY
2,-122.24,37.85,52,1467,190.0,496,177,7.2574,352100,NEAR BAY
3,-122.25,37.85,52,1274,235.0,558,219,5.6431,341300,NEAR BAY
4,-122.25,37.85,52,1627,280.0,565,259,3.8462,342200,NEAR BAY


In [636]:
snowpark_df= session.write_pandas(housing, "HOUSING_DATA")

In [637]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [638]:
#MODEL BUILDING

In [639]:
import snowflake.snowpark
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import sproc

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

import pandas as pd
import numpy as np
import datetime
import io
import joblib

In [640]:
connection_parameter={
    "account": "xw90760.ap-southeast-1",
    "user": "MUNEERSAYYED",
    "password": "Muneer@786",  # Ensure the password is correct and secure
    "database": "SNOWFLAKE_MUNEER",
    "schema": "TEST_SCHEMA",
    "warehouse": "compute_wh",
    "role": "Accountadmin" 
}

session1=Session.builder.configs(connection_parameter).create()

In [641]:
session1.add_packages('snowflake-snowpark-python','scikit-learn','pandas','numpy','cachetools')

The version of package 'snowflake-snowpark-python' in the local environment is 1.21.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'cachetools' in the local environment is 5.5.0, which does not fit the criteria for the requirement 'cachetools'. Your UDF might not work when the package version is different between the server and your local environment.


In [642]:
snowpark_df= session1.write_pandas(housing, "HOUSING_DATA")

In [643]:
#CREATING STAGES IN SNOWFLAKE

In [644]:
query="""CREATE OR REPLACE STAGE ml_training_stage
DIRECTORY=(ENABLE=TRUE)
COPY_OPTIONS=(ON_ERROR='SKIP_FILE')"""
session1.sql(query).collect()

query="""CREATE OR REPLACE STAGE ml_output_stage
COPY_OPTIONS=(ON_ERROR='SKIP_FILE')"""
session1.sql(query).collect()

[Row(status='Stage area ML_OUTPUT_STAGE successfully created.')]

In [645]:
#PREPROCESSING MODEL TRAINING AND FITTING

In [646]:
def save_file(session1,model,path):
    input_stream=io.BytesIO()
    joblib.dump(model,input_stream)
    session1._conn._cursor.upload_stream(input_stream,path)
    return "successfully created file: "+path

In [647]:
def train_model(session1: snowflake.snowpark.Session) -> float:
    # Load the data
    snowdf = session1.table("HOUSING_DATA")
    
    # Split the data into training and test sets
    snowdf_train, snowdf_test = snowdf.random_split([0.8, 0.2], seed=82)  # Corrected the syntax here
    
    # Save the split data as tables
    snowdf_train.write.save_as_table("HOUSING_TRAIN", mode="overwrite")
    snowdf_test.write.save_as_table("HOUSING_TEST", mode="overwrite")
    
    # Convert Snowpark DataFrames to Pandas DataFrames
    housing_train = snowdf_train.to_pandas().drop("MEDIAN_HOUSE_VALUE", axis=1)
    housing_train_labels = snowdf_train.to_pandas()["MEDIAN_HOUSE_VALUE"]
    housing_test = snowdf_test.to_pandas().drop("MEDIAN_HOUSE_VALUE", axis=1)
    housing_test_labels = snowdf_test.to_pandas()["MEDIAN_HOUSE_VALUE"]
    
    # Separate numerical and categorical columns
    housing_num = housing_train.drop("OCEAN_PROXIMITY", axis=1)
    
    # Create pipelines for preprocessing
    num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])
    
    num_attribs = list(housing_num)
    cat_attribs = ["OCEAN_PROXIMITY"]
    
    preprocessor = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs)
    ])
    
    # Full pipeline with model
    full_pipeline = Pipeline([
        ("preprocess", preprocessor),
        ("model", RandomForestRegressor(n_estimators=100, random_state=22)),
    ])
    
    # Train the model
    full_pipeline.fit(housing_train, housing_train_labels)
    
    # Save the model
    save_file(session1, full_pipeline, "@ml_output_stage/housing_price_reg.joblib")
    
    # Make predictions and calculate RMSE
    housing_predictions = full_pipeline.predict(housing_test)
    lin_mse = mean_squared_error(housing_test_labels, housing_predictions)
    lin_rmse = np.sqrt(lin_mse)
    
    return lin_rmse


In [648]:
#CREATING SPROC

In [649]:
train_model_sp = sproc(func=train_model,
    name='ml_model_hp',
    stage_location='@ml_training_stage',
    is_permanent=True,
    replace=True,
    session=session1
)

Got error object of type 'NoneType' has no len() when trying to read default values from function: <function train_model at 0x000001D0B156E820>. Proceeding without creating optional arguments


In [650]:
train_model_sp(session=session1)

27726.38707804281

In [651]:
#SCHEDULE A TASK TO PERFORM MODEL TRAINING EVERY MONDAY AT 10 AM UTC

In [652]:
sql="""CREATE OR REPLACE TASK schedule_model_training
WAREHOUSE=COMPUTE_WH
SCHEDULE='USING CRON*10** MON UTC'
AS
call ml_model_hp();"""
session1.sql(sql)

In [653]:
#CREATE STAGE FOR MODEL SERVING UDF

In [654]:
query="create or replace stage model_serving_stage"
"copy_options=(on_error='skip_files')"
session1.sql(query).collect()

[Row(status='Stage area MODEL_SERVING_STAGE successfully created.')]

In [655]:
#CREATE UDF FOR PREDICTION SERVING

In [663]:
import sys
import cachetools
import os
import joblib
import pandas as pd
from snowflake.snowpark.functions import udf
from snowflake.snowpark import Session

# Assuming 'session1' is an active Snowpark session
session1.add_import("@ml_output_stage/housing_price_reg.joblib")

@cachetools.cached(cache={})
def read_file(filename):
    import_dir = sys._xoptions.get("snowflake_import_directory")
    if import_dir:
        with open(os.path.join(import_dir, filename), 'rb') as file:
            m = joblib.load(file)
            return m

features = ['LONGITUDE', 'LATITUDE', 'HOUSING_MEDIAN_AGE', 'TOTAL_ROOMS', 'TOTAL_BEDROOMS', 'POPULATION', 'HOUSEHOLDS', 'MEDIAN_INCOME', 'OCEAN_PROXIMITY']

@udf(name="predict_house_value",
     is_permanent=True,
     replace=True,
     stage_location='@model_serving_stage',
     session=session1,  # Specify the session explicitly
     packages=['pandas', 'joblib', 'cachetools','scikit-learn'])  # Include necessary packages
def predict_house_value(LONGITUDE: float, LATITUDE: float, HOUSING_MEDIAN_AGE: float, TOTAL_ROOMS: float,
                        TOTAL_BEDROOMS: float, POPULATION: float, HOUSEHOLDS: float, MEDIAN_INCOME: float,
                        OCEAN_PROXIMITY: str) -> float:
    m = read_file('housing_price_reg.joblib')
    row = pd.DataFrame([locals()], columns=features)
    return m.predict(row)[0]


Got error object of type 'NoneType' has no len() when trying to read default values from function: <function predict_house_value at 0x000001D0B2BA9550>. Proceeding without creating optional arguments
The version of package 'cachetools' in the local environment is 5.5.0, which does not fit the criteria for the requirement 'cachetools'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn'. Your UDF might not work when the package version is different between the server and your local environment.


In [664]:
#RUN PREDICTIONS & MODEL SERVING

In [665]:
from snowflake.snowpark import functions as F
snowdf_test=session1.table("HOUSING_TEST")
input=snowdf_test.drop("MEDIAN_HOUSE_VALUE")

snowdf_results=snowdf_test.select(
    predict_house_value(*input).alias("predicted_values"),(F.col("MEDIAN_HOUSE_VALUE")).alias("actual_value")).limit(20)

snowdf_results.to_pandas()

,PREDICTED_VALUES,ACTUAL_VALUE
0,141170.00,140000.0
1,116557.00,110400.0
2,203602.00,191400.0
3,219942.00,188800.0
4,89831.00,87500.0
5,174585.05,75000.0
6,82437.00,75000.0
7,125752.03,112500.0
8,183338.13,137500.0
9,249404.01,257800.0


In [666]:
#if we want to load this predicion results to table 

In [667]:
snowdf_results.write.mode("overwrite").save_as_table("PREDICTION_RESULTS")`